<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №2</center>

<center>«Обработка данных с использование Kafka»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Выполнил: Фамилия И.О.
<br>613х-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2025</center>


### Экспорт необходимых модулей

In [1]:
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.errors import KafkaError
from kafka.structs import TopicPartition

from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions


### Определение списка серверов kafka

In [2]:
bootstrap_servers = 'localhost:9092'

## Реализация схемы с одним обработчиком (one consumer)

### Создание топика для схемы *one consumer*

In [3]:
admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

topic1 = 'rides1-events'

topic_list = []
topic_list.append(NewTopic(name=topic1, num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

print(admin_client.list_topics())

admin_client.close()

['rides1-events']


### Реализация источника данных (producer)

1. Создаём экземпляр класса KafkaProducer (не забываем определить сериализаторы для key- и value-составляющих в сообщении)
2. Подключаем доступ на чтение из файла *riders.csv*
3. Читаем и пропускаем первую строку-заголовок
4. В цикле считываем строки из csv-файла и отправлем их в топик
5. Завершаем работу объекта KafkaProducer

In [4]:
producer = KafkaProducer(bootstrap_servers=[bootstrap_servers],
                        key_serializer= lambda key: key.encode('utf-8'),
                        value_serializer= lambda value: value.encode('utf-8'))


In [5]:
import csv

csv_file = "D:/projects/python/bigdata_lr0/maga_lab2/rides.csv"

with open(csv_file, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = next(reader)
    print(f"Заголовок: {header}")
    for row in reader:
        line = ",".join(row)
        producer.send(topic1, key="key", value=line)

producer.flush()

Заголовок: ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge']


In [6]:
producer.close()

### Реализация обработчика данных (consumer)

1. Создаём экземпляр класса KafkaConsumer (не забываем в параметрах определить параметр *consumer_timeout_ms* и десериализаторы для key- и value-составляющих в сообщении)
2. Создаём массивы для накопления значений trip_count, passenger_count, trip_distance, total_amount с нулевыми значениями и соответствующим типом данных
3. В цикле получения сообщений объекта KafkaConsumer:
   * разделяем получаемую value-строку на отдельные элементы,
   * выделяем значение VendorID или используем key, если источник передаёт его в сообщении,
   * обновляем значения в массивах-накопителях, используя (VendorID - 1) как индекс соответствующего массива 
4. Распечатываем отдельно для каждого перевозчика:
   * количество поездок(trip_count),
   * число перевезённых пассажиров(passenger_count),
   * расстояние поездок(trip_distance)
   * сумму оплаты(total_amount)
5. Завершаем работу объекта KafkaProducer

In [7]:
consumer = KafkaConsumer(topic1,
                         auto_offset_reset='earliest',
                         consumer_timeout_ms=2000,
                         key_deserializer= lambda key: key.decode('utf-8'),
                         value_deserializer= lambda value: value.decode('utf-8'),
                         bootstrap_servers=[bootstrap_servers])


In [8]:
trip_count = [0, 0]
passenger_count = [0, 0]
trip_distance = [0.0, 0.0]
total_amount = [0.0, 0.0]

for msg in consumer:
    parts = msg.value.split(",")
    vendor_id = int(parts[0])
    idx = vendor_id - 1

    trip_count[idx] += 1
    passenger_count[idx] += int(parts[3])
    trip_distance[idx] += float(parts[4])
    total_amount[idx] += float(parts[16])

for vendor in (1, 2):
    idx = vendor - 1
    print(f"\nПеревозчик VendorID = {vendor}:")
    print(f"Количество поездок: {trip_count[idx]}")
    print(f"Число пассажиров: {passenger_count[idx]}")
    print(f"Дистанция поездок: {trip_distance[idx]:.2f} миль")
    print(f"Общая сумма оплаты: {total_amount[idx]:.2f} $")


Перевозчик VendorID = 1:
Количество поездок: 87
Число пассажиров: 101
Дистанция поездок: 394.30 миль
Общая сумма оплаты: 1976.63 $

Перевозчик VendorID = 2:
Количество поездок: 179
Число пассажиров: 242
Дистанция поездок: 744.65 миль
Общая сумма оплаты: 3709.17 $


In [9]:
consumer.close()

## Реализация схемы с двумя обработчиками (two consumers launched in a group)

### Создание топика для схемы *two consumer*


!!! Не забываем указать количество разделов с помощью параметра *num_partitions=2*

In [10]:
admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

topic2 = 'rides2-events'

topic_list = []
topic_list.append(NewTopic(name=topic2, num_partitions=2, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

print(admin_client.list_topics())

admin_client.close()

['rides2-events', 'rides1-events']


### Реализация источника данных (producer)

1. Создаём экземпляр класса KafkaProducer (не забываем определить сериализаторы для key- и value-составляющих в сообщении)
2. Подключаем доступ на чтение из файла *riders.csv*
3. Читаем и пропускаем первую строку-заголовок
4. В цикле считываем строки из csv-файла:
    * преобразуем строку в список элементов (используем метод split(','))
    * выделяем значение VendorID
    * отправлем сообщение в топик с key=*VendorID* и value=*cтрока файла*
6. Завершаем работу объекта KafkaProducer

In [11]:
producer2 = KafkaProducer(bootstrap_servers=[bootstrap_servers],
                        key_serializer= lambda key: key.encode('utf-8'),
                        value_serializer= lambda value: value.encode('utf-8'))

In [12]:
import csv
import time

csv_file = "D:/projects/python/bigdata_lr0/maga_lab2/rides.csv"

with open(csv_file, "r", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        line = ",".join(row)
        record = producer2.send(topic2, key=row[0], value=line)
        result = record.get(timeout=60)
        print(result.partition, result.offset, row[0], row[3], row[4], row[16])
        time.sleep(0.5)

producer2.flush()

1 0 1 1 1.50 9.3
1 1 1 1 9.50 27.8
0 0 2 1 5.85 22.3
0 1 2 1 1.90 14.16
0 2 2 1 1.25 7.8
1 2 1 1 9.70 33.8
0 3 2 1 5.27 26.39
0 4 2 1 1.32 8.8
0 5 2 1 .73 10.12
0 6 2 1 18.65 66.36
1 3 1 2 8.00 28.3
0 7 2 3 4.97 19.8
0 8 2 1 .00 66.36
0 9 2 1 .57 8.3
0 10 2 1 .00 8.19
0 11 2 1 9.41 36.3
1 4 1 1 2.80 5.8
1 5 1 2 13.50 39.8
0 12 2 1 .48 9.36
0 13 2 1 1.67 12.09
0 14 2 1 3.75 21.19
0 15 2 1 3.45 18.96
1 6 1 2 6.70 30.8
1 7 1 2 1.70 11.3
1 8 1 1 1.10 10.8
0 16 2 1 1.36 11.3
0 17 2 1 2.01 13.3
0 18 2 3 .77 8.3
1 9 1 1 1.10 8.8
1 10 1 1 3.50 15.8
0 19 2 1 .94 11.76
0 20 2 1 1.50 10.8
0 21 2 1 1.86 12.3
0 22 2 1 7.85 41.9
1 11 1 1 1.40 10.8
1 12 1 1 .90 11.15
0 23 2 1 2.33 12.96
0 24 2 1 1.41 8.8
0 25 2 1 5.03 24.36
0 26 2 1 1.44 11.3
0 27 2 1 9.30 35.42
0 28 2 1 .75 6.1
0 29 2 1 4.44 18.3
0 30 2 2 2.39 14.76
0 31 2 3 6.42 24.3
0 32 2 1 3.77 16.83
0 33 2 1 2.89 13.8
0 34 2 1 5.07 23.3
0 35 2 3 2.55 13.3
0 36 2 4 1.95 12.3
0 37 2 1 12.66 40.55
0 38 2 1 11.60 39.3
0 39 2 1 10.27 49.28
0 40 2 1 

In [13]:
producer2.close()

## Полезные утилиты

### Просмотр списка топиков

In [ ]:
from kafka.errors import KafkaError
from kafka.structs import TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions

bootstrap_servers = 'localhost:9092'

admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

print(admin_client.list_topics())

admin_client.close()

### Удаление топика

In [ ]:
from kafka.errors import KafkaError
from kafka.structs import TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions

bootstrap_servers = 'localhost:9092'

# ниже введите имя топика из списка, распечатанного предыдущей ячейкой
topic_for_del = 'rides1-events'

admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
admin_client.delete_topics([topic_for_del])

print(admin_client.list_topics())

admin_client.close()

### Просмотр списка групп

In [ ]:
from kafka.errors import KafkaError
from kafka.structs import TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions

bootstrap_servers = 'localhost:9092'

admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

print(admin_client.list_consumer_groups())

admin_client.close()

### Просмотр описания заданной группы (в данном случае – *rides2*)

In [ ]:
from kafka.errors import KafkaError
from kafka.structs import TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions

bootstrap_servers = 'localhost:9092'

admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

print(admin_client.describe_consumer_groups(group_ids=['rides2']))

admin_client.close()